# Proyecto Teoría de la Simulación


# Amílcar José Moncada
# 20211002386

## Trabajaré en un Dataset el cual habla del rendimiento de estudiantes en examenes

In [308]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.stats import truncnorm

In [309]:
dataframe = pd.read_csv('studentsperformanceinexams.csv')

In [310]:
dataframe

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68
...,...,...,...,...,...,...,...,...
995,male,group C,high school,standard,none,73,70,65
996,male,group D,associate's degree,free/reduced,completed,85,91,92
997,female,group C,some high school,free/reduced,none,32,35,41
998,female,group C,some college,standard,none,73,74,82


### El dataset contiene información como el genero, raza, nivel de educación de los padres, si el estudiante almuerza correctamente, si el estudiante a tomado un curso de preparación (todos los datos anteriores son categoricos) y por ultimo las notas en las clases de matematicas, lectura y escritura. 

In [311]:
dataframe.dtypes

gender                         object
race/ethnicity                 object
parental level of education    object
lunch                          object
test preparation course        object
math score                      int64
reading score                   int64
writing score                   int64
dtype: object

### Modificamos los nombres de las columnas para poder manipular las columnas de manera mas sencilla.

In [312]:
dataframe.columns = ["gender" , "group" , "par_edu" , "lunch" , "test" , "math" , "reading" , "writing"]

In [313]:
dataframe.head(2)


,gender,group,par_edu,lunch,test,math,reading,writing
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55


### Almacenaremos de manera dinamica las columnas segun su tipo de dato

In [314]:
categoricos = []
numericos = []

for i in dataframe.columns:
    if dataframe[i].dtypes == 'int64':
        numericos.append(i)
    else:
        categoricos.append(i)

##### Eliminare Lunch y el group (raza) ya que no es un valor que me es relevante, porque me quiero enfocar totalmente en la parte academica.

In [315]:
categoricos.remove("group")
categoricos.remove("lunch")


### Campos Categoricos

In [316]:
categoricos

['gender', 'par_edu', 'test']

### Campos Numericos

In [317]:
numericos

['math', 'reading', 'writing']

### Agrupamos información para obtener las distribuciones estadisticas

In [318]:
frecuencias = (
    dataframe.groupby(categoricos, as_index=False).size()
)


In [319]:
frecuencias

,gender,par_edu,test,size
0,female,associate's degree,completed,27
1,female,associate's degree,none,54
2,female,bachelor's degree,completed,21
3,female,bachelor's degree,none,37
4,female,high school,completed,28
5,female,high school,none,66
6,female,master's degree,completed,12
7,female,master's degree,none,30
8,female,some college,completed,37
9,female,some college,none,63


### Guardamos el porcentaje

In [320]:
frecuencias["size"] = frecuencias["size"] / dataframe.shape[0]

In [321]:
frecuencias

,gender,par_edu,test,size
0,female,associate's degree,completed,0.027
1,female,associate's degree,none,0.054
2,female,bachelor's degree,completed,0.021
3,female,bachelor's degree,none,0.037
4,female,high school,completed,0.028
5,female,high school,none,0.066
6,female,master's degree,completed,0.012
7,female,master's degree,none,0.030
8,female,some college,completed,0.037
9,female,some college,none,0.063


### Ahora se identifica de manera dinamica los Upper Bound y Lower Bound

In [322]:
Upper_Lower = {}
columnas = []
for i in numericos:
    Upper_Lower[i] = [
        ( 'lower', lambda x: x.quantile(.25) - 1.5 * (x.quantile(.75) - x.quantile(.25) )),
        ( 'upper', lambda x: x.quantile(.75) + 1.5 * (x.quantile(.75) - x.quantile(.25) ))
    ]
    columnas.append(f"{i}_lower")
    columnas.append(f"{i}_upper")

dataframe_resultado = dataframe.groupby(categoricos, as_index= False).agg(Upper_Lower)

In [323]:
dataframe_resultado.columns = categoricos + columnas

In [324]:
dataframe_resultado


,gender,par_edu,test,math_lower,math_upper,reading_lower,reading_upper,writing_lower,writing_upper
0,female,associate's degree,completed,15.750,117.750,35.500,119.500,42.000,118.000
1,female,associate's degree,none,28.750,104.750,35.500,109.500,31.250,109.250
2,female,bachelor's degree,completed,24.000,112.000,32.500,124.500,32.000,128.000
3,female,bachelor's degree,none,25.500,109.500,32.500,116.500,34.000,114.000
4,female,high school,completed,26.125,103.125,34.000,116.000,35.500,121.500
5,female,high school,none,22.875,97.875,28.375,105.375,28.875,103.875
6,female,master's degree,completed,42.500,122.500,43.125,134.125,59.375,124.375
7,female,master's degree,none,24.875,113.875,30.125,121.125,32.625,119.625
8,female,some college,completed,24.000,104.000,37.500,113.500,37.000,117.000
9,female,some college,none,15.000,107.000,31.500,107.500,27.000,115.000


### Realizamos Join con los valores de los examenes con los Upper y Lower Bounds

In [325]:
dataframe_bound = dataframe.merge(
    dataframe_resultado
    , on = categoricos
    , how = "inner"
)

In [326]:
dataframe_bound

,gender,group,par_edu,lunch,test,math,reading,writing,math_lower,math_upper,reading_lower,reading_upper,writing_lower,writing_upper
0,male,group A,high school,standard,completed,67,67,63,46.500,98.500,39.000,103.000,45.000,93.000
1,female,group D,some high school,free/reduced,none,40,59,55,11.500,95.500,32.000,96.000,18.000,106.000
2,male,group E,some college,free/reduced,none,59,60,50,17.000,113.000,13.500,105.500,7.500,107.500
3,male,group B,high school,standard,none,77,78,68,30.500,102.500,32.000,96.000,23.250,93.250
4,male,group E,associate's degree,standard,completed,78,73,68,36.375,111.375,43.875,106.875,46.375,103.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,male,group C,high school,standard,none,73,70,65,30.500,102.500,32.000,96.000,23.250,93.250
996,male,group D,associate's degree,free/reduced,completed,85,91,92,36.375,111.375,43.875,106.875,46.375,103.375
997,female,group C,some high school,free/reduced,none,32,35,41,11.500,95.500,32.000,96.000,18.000,106.000
998,female,group C,some college,standard,none,73,74,82,15.000,107.000,31.500,107.500,27.000,115.000


### De forma dinamica eliminamos los datos atipicos existentes (siguiendo de ejemplo la manera que se hizo en el video)

In [327]:
for i in numericos:
    print( f"Cantidad de {dataframe_bound.shape[0]} registros")
    print( f"Se esta procesando la columna {i}")

    dataframe_bound = dataframe_bound.loc[
        ( dataframe_bound[i] > dataframe_bound[f"{i}_lower"]) &
        ( dataframe_bound[i] < dataframe_bound[f"{i}_upper"]),:
    ]

print(f"Limpieza culminada correctamente, con una cantidad final de {dataframe_bound.shape[0]} registros")

Cantidad de 1000 registros
Se esta procesando la columna math
Cantidad de 995 registros
Se esta procesando la columna reading
Cantidad de 990 registros
Se esta procesando la columna writing
Limpieza culminada correctamente, con una cantidad final de 988 registros


In [328]:
porcentaje = (1 - dataframe_bound.shape[0]/1000)*100
print( f"Eliminamos un {porcentaje} % de los datos del dataset.")

Eliminamos un 1.200000000000001 % de los datos del dataset.


In [329]:
dataframe_bound

,gender,group,par_edu,lunch,test,math,reading,writing,math_lower,math_upper,reading_lower,reading_upper,writing_lower,writing_upper
0,male,group A,high school,standard,completed,67,67,63,46.500,98.500,39.000,103.000,45.000,93.000
1,female,group D,some high school,free/reduced,none,40,59,55,11.500,95.500,32.000,96.000,18.000,106.000
2,male,group E,some college,free/reduced,none,59,60,50,17.000,113.000,13.500,105.500,7.500,107.500
3,male,group B,high school,standard,none,77,78,68,30.500,102.500,32.000,96.000,23.250,93.250
4,male,group E,associate's degree,standard,completed,78,73,68,36.375,111.375,43.875,106.875,46.375,103.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,male,group C,high school,standard,none,73,70,65,30.500,102.500,32.000,96.000,23.250,93.250
996,male,group D,associate's degree,free/reduced,completed,85,91,92,36.375,111.375,43.875,106.875,46.375,103.375
997,female,group C,some high school,free/reduced,none,32,35,41,11.500,95.500,32.000,96.000,18.000,106.000
998,female,group C,some college,standard,none,73,74,82,15.000,107.000,31.500,107.500,27.000,115.000


### Quitamos las columnas de Lower y Upper, ya que despues de la impieza no las necesitamos

In [330]:
dataframe_clean = dataframe_bound.loc[ :, categoricos + numericos]

In [331]:
dataframe_clean

,gender,par_edu,test,math,reading,writing
0,male,high school,completed,67,67,63
1,female,some high school,none,40,59,55
2,male,some college,none,59,60,50
3,male,high school,none,77,78,68
4,male,associate's degree,completed,78,73,68
...,...,...,...,...,...,...
995,male,high school,none,73,70,65
996,male,associate's degree,completed,85,91,92
997,female,some high school,none,32,35,41
998,female,some college,none,73,74,82


### Preparación del resumen

In [332]:
summ_agregaciones = {}
columnas_agr = []

for i in numericos:
    summ_agregaciones[i] = ["min", "max", "mean", "std"]
    columnas_agr.append( f"{i}_min")
    columnas_agr.append( f"{i}_max")
    columnas_agr.append( f"{i}_mean")
    columnas_agr.append( f"{i}_std")

In [333]:
summ_agregaciones

{'math': ['min', 'max', 'mean', 'std'],
 'reading': ['min', 'max', 'mean', 'std'],
 'writing': ['min', 'max', 'mean', 'std']}

In [334]:
dataframe_summ = dataframe_clean.groupby( categoricos , as_index= False).agg(summ_agregaciones)

In [335]:
dataframe_summ.columns = categoricos + columnas_agr


In [336]:
dataframe_summ

,gender,par_edu,test,math_min,math_max,math_mean,math_std,reading_min,reading_max,reading_mean,reading_std,writing_min,writing_max,writing_mean,writing_std
0,female,associate's degree,completed,41,95,68.703704,15.585533,56,100,77.888889,13.051181,53,100,80.444444,13.238444
1,female,associate's degree,none,32,92,66.867925,13.334510,41,100,73.283019,12.504426,39,100,71.811321,13.166181
2,female,bachelor's degree,completed,43,100,69.047619,15.493470,53,100,79.809524,13.984345,60,100,81.857143,13.499206
3,female,bachelor's degree,none,37,89,66.378378,13.433688,53,100,74.216216,12.277204,48,100,74.513514,12.722730
4,female,high school,completed,43,90,65.571429,12.784332,48,96,75.250000,12.819617,53,97,77.428571,12.755329
5,female,high school,none,30,93,60.378788,14.268594,32,100,68.060606,14.726093,35,96,65.696970,13.911777
6,female,master's degree,completed,57,100,79.833333,14.121765,68,100,86.916667,12.228568,74,100,90.166667,9.703170
7,female,master's degree,none,40,91,68.133333,14.214450,50,100,76.766667,14.277263,44,100,77.300000,15.454382
8,female,some college,completed,26,100,63.694444,15.399108,44,100,75.333333,13.401492,46,100,77.083333,13.004120
9,female,some college,none,38,91,63.475410,13.679189,44,100,71.491803,12.964854,47,96,70.918033,12.361628


### Comenzamos el proceso de expansión del Dataset

In [337]:
expansión = 200000

In [338]:
valores = np.random.choice ( frecuencias.index, size=expansión, p=frecuencias["size"].to_list() )

In [339]:
def trunc(min, max, media, desv_std, size):
    a, b = (min - media) / desv_std, (max - media) / desv_std
    muestras = truncnorm.rvs( a, b, loc=media, scale=desv_std, size=size)
    return muestras

In [340]:
dataframe_extendido = pd.DataFrame()

for h in frecuencias.index:
    conf = {
        "valor": valores[valores == h]
    }
    for i in numericos:
        
        conf[i] = trunc(
            dataframe_summ.iloc[h][f"{i}_min"],
            dataframe_summ.iloc[h][f"{i}_max"],
            dataframe_summ.iloc[h][f"{i}_mean"],
            dataframe_summ.iloc[h][f"{i}_std"],
            len(valores[valores == h])
        )

    temporal = pd.DataFrame(conf)
    dataframe_extendido = pd.concat([dataframe_extendido, temporal])
 

In [341]:
dataframe_extendido

,valor,math,reading,writing
0,0,66.349095,97.457294,76.174302
1,0,71.368743,56.309546,61.556761
2,0,65.583542,86.954808,93.318983
3,0,68.947989,63.747881,61.391793
4,0,50.201269,71.653795,66.998794
...,...,...,...,...
10983,23,66.304101,35.895153,40.864774
10984,23,89.485592,81.926025,47.582048
10985,23,72.365068,36.425315,71.963164
10986,23,79.008101,62.386017,58.691082


In [342]:

frecuencias["valor"] = frecuencias.index

In [343]:
frecuencias

,gender,par_edu,test,size,valor
0,female,associate's degree,completed,0.027,0
1,female,associate's degree,none,0.054,1
2,female,bachelor's degree,completed,0.021,2
3,female,bachelor's degree,none,0.037,3
4,female,high school,completed,0.028,4
5,female,high school,none,0.066,5
6,female,master's degree,completed,0.012,6
7,female,master's degree,none,0.030,7
8,female,some college,completed,0.037,8
9,female,some college,none,0.063,9


In [344]:
dataframe_final = frecuencias.loc[: , ["valor"] + categoricos].merge(
    dataframe_extendido,
    on="valor",
    how="inner"
    )

In [345]:
dataframe_final = dataframe_final.drop(columns=["valor"])


In [346]:
dataframe_final

,gender,par_edu,test,math,reading,writing
0,female,associate's degree,completed,66.349095,97.457294,76.174302
1,female,associate's degree,completed,71.368743,56.309546,61.556761
2,female,associate's degree,completed,65.583542,86.954808,93.318983
3,female,associate's degree,completed,68.947989,63.747881,61.391793
4,female,associate's degree,completed,50.201269,71.653795,66.998794
...,...,...,...,...,...,...
199995,male,some high school,none,66.304101,35.895153,40.864774
199996,male,some high school,none,89.485592,81.926025,47.582048
199997,male,some high school,none,72.365068,36.425315,71.963164
199998,male,some high school,none,79.008101,62.386017,58.691082


### Resumen estadistico del dataset original

In [350]:
dataframe_clean.describe()

,math,reading,writing
count,988.000000,988.000000,988.000000
mean,66.786437,69.368421,68.093117
std,15.014344,14.417424,15.317294
min,25.000000,27.000000,24.000000
25%,56.000000,60.000000,58.000000
50%,67.000000,70.000000,68.000000
75%,77.000000,79.000000,79.000000
max,100.000000,100.000000,100.000000


### Resumen estadistico del dataset simulado

In [348]:
dataframe_final.describe()

,math,reading,writing
count,200000.000000,200000.000000,200000.000000
mean,66.683789,69.186519,67.867331
std,13.657974,13.208054,13.989410
min,25.013912,27.034426,24.040977
25%,57.129390,60.177881,58.290224
50%,66.985156,69.636096,68.250211
75%,76.525605,78.688947,77.952082
max,99.996165,99.998491,99.994618


##### Podemos observar que hemos realizado una simulación correcta dado que estamos teniendo un alto porcentaje de efectividad con lo cercanos que estan las cantidades obtenidas junto a las originales.

##### Se observa tambien que el dataset original traia un numero bajo de valores atipicos (12) el cual facilitó el desarrollo del proyecto, tambien esto beneficia la simulación, ya que nos acerca aún mas a los valores que buscamos simular. 